In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Attention


num_samples = 10000
latent_dim = 256


data_path = 'path_to_your_dataset.txt'


input_texts = []
target_texts = []
input_vocab = set()
target_vocab = set()


with open(data_path, 'r', encoding='utf-8') as file:
    lines = file.read().split('\n')
    for line in lines[:num_samples]:
        input_text, target_text = line.split('\t')
        input_texts.append(input_text)
        target_texts.append('\t' + target_text + '\n')
        for word in input_text.split():
            input_vocab.add(word)
        for word in target_text.split():
            target_vocab.add(word)

input_vocab = sorted(list(input_vocab))
target_vocab = sorted(list(target_vocab))
num_encoder_tokens = len(input_vocab)
num_decoder_tokens = len(target_vocab)
input_token_index = dict([(word, i) for i, word in enumerate(input_vocab)])
target_token_index = dict([(word, i) for i, word in enumerate(target_vocab)])


encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t, input_token_index[word]] = 1.0
    for t, word in enumerate(target_text.split()):
        decoder_input_data[i, t, target_token_index[word]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.0


encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)


model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100, validation_split=0.2)


encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


def translate(input_sentence):

    input_sequence = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    for t, word in enumerate(input_sentence.split()):
        input_sequence[0, t, input_token_index[word]] = 1.0


    states_value = encoder_model.predict(input_sequence)


    target_sequence = np.zeros((1, 1, num_decoder_tokens), dtype='float32')
    target_sequence[0, 0, target_token_index['\t']] = 1.0

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:

        outputs, h, c = decoder_model.predict([target_sequence] + states_value)


        sampled_token_index = np.argmax(outputs[0, -1, :])
        sampled_char = target_vocab[sampled_token_index]

        decoded_sentence += sampled_char


        if sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True


        target_sequence = np.zeros((1, 1, num_decoder_tokens), dtype='float32')
        target_sequence[0, 0, sampled_token_index] = 1.0
        states_value = [h, c]

    return decoded_sentence


input_sentence = 'your input sentence in the source language'
print('Input sentence:', input_sentence)
print('Translated sentence:', translate(input_sentence))